<a href="https://colab.research.google.com/github/dsfdev2023/studentEvaluatiuon/blob/main/HR_KPIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1 & 2: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 3: Read the Excel file
import pandas as pd

file_path = '/content/drive/MyDrive/1KPIS HR /updated_file.xlsx' # Change to your file path
df = pd.read_excel(file_path)


Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [2]:
# Step 4: Preprocess the data
def convert_to_minutes(time_val):
    if pd.isnull(time_val):
        return 0

    # Check if the value is an integer
    if isinstance(time_val, int):
        return time_val

    # Check if the value is a string
    elif isinstance(time_val, str):
        hours, minutes = map(int, time_val.split(':'))
        return hours * 60 + minutes

    # For datetime.time objects
    elif hasattr(time_val, 'hour'):
        return time_val.hour * 60 + time_val.minute

    # Fallback for any other type
    else:
        return 0

df['real Presence'] = df['real Presence'].apply(convert_to_minutes)
df['Pause'] = df['Pause'].apply(convert_to_minutes)

# Step 5: Perform aggregations
grouped = df.groupby('Ligne').agg({
    'real Presence': 'sum',
    'Breaks': 'sum',
    'Pause': 'sum'
})

# Import Plotly
import plotly.express as px

# Ensure that your data frame 'grouped' is reset to include 'Ligne' as a column
grouped = grouped.reset_index()

# Real Presence by Ligne - Interactive Graph with Different Colors
fig_real_presence = px.bar(grouped, x='Ligne', y='real Presence', color='Ligne',
                           labels={'real Presence': 'Real Presence (Minutes)'},
                           title='Sum of Real Presence by Ligne')
fig_real_presence.show()

# Breaks by Ligne - Interactive Graph with Different Colors
fig_breaks = px.bar(grouped, x='Ligne', y='Breaks', color='Ligne',
                    labels={'Breaks': 'Breaks (Count)'},
                    title='Sum of Breaks by Ligne')
fig_breaks.show()

# Pause by Ligne - Interactive Graph with Different Colors
fig_pause = px.bar(grouped, x='Ligne', y='Pause', color='Ligne',
                   labels={'Pause': 'Pause (Minutes)'},
                   title='Sum of Pause by Ligne')
fig_pause.show()


# **GRAPHS FOR 'BAKER' LIGNE**

In [4]:
# Step 1: Filter the DataFrame for 'Ligne' equal to 'BAKER'
df_baker = df[df['Ligne'] == 'BAKER']

# Step 2: Create Histograms

# Pause Histogram
fig_pause_baker = px.histogram(df_baker, x='Matricule', y='Pause',
                               title='Pause Time for BAKER Matricule',
                               labels={'Pause': 'Pause (Minutes)'},
                               category_orders={"Matricule": df_baker.sort_values(by='Pause', ascending=False)['Matricule'].unique()})
fig_pause_baker.update_layout(bargap=0.2)
fig_pause_baker.show()

# Breaks Histogram
fig_breaks_baker = px.histogram(df_baker, x='Matricule', y='Breaks',
                                title='Breaks Count for BAKER Matricule',
                                labels={'Breaks': 'Breaks (Count)'},
                                category_orders={"Matricule": df_baker.sort_values(by='Breaks', ascending=False)['Matricule'].unique()})
fig_breaks_baker.update_layout(bargap=0.2)
fig_breaks_baker.show()

# Real Presence Histogram
fig_real_presence_baker = px.histogram(df_baker, x='Matricule', y='real Presence',
                                       title='Real Presence Time for BAKER Matricule',
                                       labels={'real Presence': 'Real Presence (Minutes)'},
                                       category_orders={"Matricule": df_baker.sort_values(by='real Presence', ascending=False)['Matricule'].unique()})
fig_real_presence_baker.update_layout(bargap=0.2)
fig_real_presence_baker.show()



# **FILTER BY SHIFT**

In [6]:
# Filter the DataFrame for 'Ligne' equal to 'BAKER'
df_baker = df[df['Ligne'] == 'BAKER']

# Function to create histogram for each metric
def create_histogram(df, metric, title, label):
    # Create a histogram for each 'Shift'
    fig = px.histogram(df, x='Matricule', y=metric, facet_col='Shift', color='Shift',
                       title=title, labels={metric: label},
                       category_orders={"Shift": df['Shift'].unique()})
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    fig.update_layout(bargap=0.2)
    return fig

# Pause Histogram
fig_pause_baker = create_histogram(df_baker, 'Pause',
                                   f'Pause Time for BAKER Matricule (Total Matricules: {df_baker["Matricule"].nunique()})',
                                   'Pause (Minutes)')
fig_pause_baker.show()

# Real Presence Histogram
fig_real_presence_baker = create_histogram(df_baker, 'real Presence',
                                           f'Real Presence Time for BAKER Matricule (Total Matricules: {df_baker["Matricule"].nunique()})',
                                           'Real Presence (Minutes)')
fig_real_presence_baker.show()

# Breaks Histogram
fig_breaks_baker = create_histogram(df_baker, 'Breaks',
                                    f'Breaks Count for BAKER Matricule (Total Matricules: {df_baker["Matricule"].nunique()})',
                                    'Breaks (Count)')
fig_breaks_baker.show()


# **Table Summary**

We will include "Matricules" of employees who have:
* Over 120 minutes of "Pause", or
* Over 2 "Breaks", or
* Less than 600 minutes of "real Presence",

But we will exclude those who have:
* More than 600 minutes of "real Presence" and fewer than 4 "Breaks".

In [14]:
# Filter the DataFrame for 'Ligne' equal to 'BAKER'
df_baker = df[df['Ligne'] == 'BAKER']

# Calculate the sums of 'Pause', 'Breaks', and 'real Presence' for each 'Matricule'
aggregated_data = df_baker.groupby('Matricule').agg({
    'Pause': 'sum',
    'Breaks': 'sum',
    'real Presence': 'sum'
}).reset_index()

# Convert 'Pause' and 'real Presence' from the hour:minute format to total minutes
aggregated_data['Pause'] = aggregated_data['Pause'].apply(convert_to_minutes)
aggregated_data['real Presence'] = aggregated_data['real Presence'].apply(convert_to_minutes)

# Conditions for filtering
condition = (
    (aggregated_data['Pause'] > 120) |
    (aggregated_data['Breaks'] > 2) |
    (aggregated_data['real Presence'] < 600)
) & ~(
    (aggregated_data['real Presence'] > 600) &
    (aggregated_data['Breaks'] < 4)
)

# Display the table
filtered_matricules = aggregated_data[condition]

sorted_filtered_matricules = filtered_matricules.sort_values(by='real Presence', ascending=True)


# Count the number of matricules in the sorted filtered table
num_matricules = sorted_filtered_matricules.shape[0]

# Print the number of matricules
print(f"Number of Employees in the table: {num_matricules}")

# Display the sorted table
sorted_filtered_matricules

Number of Employees in the table: 20


,Matricule,Pause,Breaks,real Presence
12,0599,465,6,250
18,2212,17,2,295
27,6861,115,4,344
40,7069,322,5,397
7,0432,326,8,405
28,6913,202,4,507
38,7032,183,3,529
26,6807,179,6,536
15,2200,158,3,553
17,2211,140,3,554


## 10 "Matricules" with the lowest "real Presence" and the other with the 10 "Matricules" with the highest "real Presence":

In [18]:
from IPython.display import display

# Define a function to convert minutes to "HH:MM" format
def minutes_to_hours(minutes):
    hours = minutes // 60
    minutes = minutes % 60
    return f"{hours:02d}:{minutes:02d}"

# Group by 'Matricule' and sum 'real Presence' for each 'Matricule'
aggregated_baker = df_baker.groupby('Matricule').agg({
    'real Presence': 'sum'
}).reset_index()

# Assuming 'real Presence' is already in minutes as per previous context
# If 'real Presence' is in hour:minute format, uncomment the following line
# aggregated_baker['real Presence'] = aggregated_baker['real Presence'].apply(convert_to_minutes)

# Convert 'real Presence' to "HH:MM" format and create a new column
aggregated_baker['real Presence (HH:MM)'] = aggregated_baker['real Presence'].apply(minutes_to_hours)

# Get the 10 matricules with the lowest 'real Presence'
lowest_real_presence_baker = aggregated_baker.nsmallest(10, 'real Presence')

# Get the 10 matricules with the highest 'real Presence'
highest_real_presence_baker = aggregated_baker.nlargest(10, 'real Presence')

# Display the tables with nicer formatting
print("10 Matricules with the Lowest Real Presence:")
display(lowest_real_presence_baker[['Matricule', 'real Presence', 'real Presence (HH:MM)']])

print("\n10 Matricules with the Highest Real Presence:")
display(highest_real_presence_baker[['Matricule', 'real Presence', 'real Presence (HH:MM)']])


10 Matricules with the Lowest Real Presence:


,Matricule,real Presence,real Presence (HH:MM)
12,0599,250,04:10
18,2212,295,04:55
27,6861,344,05:44
40,7069,397,06:37
7,0432,405,06:45
28,6913,507,08:27
38,7032,529,08:49
26,6807,536,08:56
15,2200,553,09:13
17,2211,554,09:14



10 Matricules with the Highest Real Presence:


,Matricule,real Presence,real Presence (HH:MM)
10,0508,721,12:01
21,2304,719,11:59
19,2219,712,11:52
33,6950,700,11:40
20,2223,698,11:38
14,2189,693,11:33
24,6627,690,11:30
47,7238,684,11:24
2,0056,679,11:19
8,0451STG,677,11:17


## Difference between "real Presence" and 600 minutes for all "Matricules" under the "BAKER" line

In [27]:
# Assuming df is your original DataFrame
# Filter the DataFrame for 'Ligne' equal to 'BAKER'
df_baker = df[df['Ligne'] == 'BAKER'].copy()

# Define a function to convert "HH:MM" format to minutes
def convert_to_minutes(time_str):
    if isinstance(time_str, str):  # Check if the input is a string
        hours, minutes = map(int, time_str.split(':'))
        return hours * 60 + minutes
    else:
        return time_str  # If it's not a string, return as is

# Convert 'real Presence' from "HH:MM" format to minutes before the aggregation
df_baker['real Presence Minutes'] = df_baker['real Presence'].apply(convert_to_minutes)

# Now, group by 'Matricule' and sum 'real Presence Minutes' for each 'Matricule'
aggregated_baker = df_baker.groupby('Matricule').agg({
    'real Presence Minutes': 'sum'
}).reset_index()

# Define a function to calculate the positive difference only if 'real Presence' is less than 600 minutes
def calculate_positive_difference(real_presence_minutes):
    return max(600 - real_presence_minutes, 0)

# Define a function to convert minutes to hours and minutes format
def minutes_to_hours(minutes):
    hours = minutes // 60
    minutes = minutes % 60
    return f"{hours} hours and {minutes:02d} minutes"

# Apply the function to calculate the positive difference from 600 minutes
aggregated_baker['Positive Difference from 600'] = aggregated_baker['real Presence Minutes'].apply(calculate_positive_difference)

# Calculate the sum total of the positive differences
sum_total_positive_difference = aggregated_baker['Positive Difference from 600'].sum()

# Convert the sum total from minutes to hours and minutes format
total_difference_hours_minutes = minutes_to_hours(sum_total_positive_difference)

# Print the sum total of the positive differences in minutes and hours:minutes format
print(f"Sum total of the positive difference from 600 minutes: {sum_total_positive_difference} minutes ({total_difference_hours_minutes})")


# Create a DataFrame to display the result as a table
results_df = pd.DataFrame({
    'Total Positive Difference from 600 Minutes': [f"{sum_total_positive_difference} minutes"],
    'Equivalent in Hours and Minutes': [total_difference_hours_minutes]
})

# Display the results as a table
display(results_df)



Sum total of the positive difference from 600 minutes: 1814 minutes (30 hours and 14 minutes)


,Total Positive Difference from 600 Minutes,Equivalent in Hours and Minutes
0,1814 minutes,30 hours and 14 minutes
